In [15]:
import itertools
import pandas as pd
import requests
from river import datasets
from river import utils
from river import evaluate
from river import forest
from river import metrics
from river import stream

url = "https://data.alpaca.markets/v1beta3/crypto/us/bars?symbols=BTC%2FUSD&timeframe=1Min&start=2023-08-01T00%3A00%3A00Z&limit=10000&sort=asc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

data = response["bars"]['BTC/USD']

column_mapping = {
    'h': "High",
    "o": "Open",
    "l": "Low",
    "c": "Close",
    "v": "Volume_(BTC)",
    't': 'Timestamp'
}

df = pd.DataFrame(data)
df.rename(columns=column_mapping, inplace=True)

df["next_minute"] = df["Close"].shift(-1)
df['target'] = (df["Close"] < df["next_minute"]).astype(int)

bitcoinData = df

bitcoinData = bitcoinData.dropna()
bitcoinData.drop(columns=['n','Volume_(BTC)', 'vw'], inplace=True)
bitcoinData.reset_index(drop=True)
bitcoinData.set_index("Timestamp", inplace=True)
bitcoinData.index = pd.to_datetime(bitcoinData.index, format='%Y-%m-%dT%H:%M:%SZ')
bitcoinData

bitcoinData["next_minute"] = bitcoinData["Close"].shift(-1)

bitcoinData['Up'] = (bitcoinData["Close"] < bitcoinData["next_minute"]).astype(int)
bitcoinData

horizons = [1,2,3,4,5,6,7,8,9,10]

predictors = []

for horizon in horizons:
    rolling_averages = bitcoinData.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    bitcoinData[ratio_column] = bitcoinData["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    
    bitcoinData[trend_column] = bitcoinData.shift(1).rolling(horizon).sum()["Up"]
    
    predictors += [ratio_column, trend_column]

bitcoinData.dropna(inplace=True)

x = bitcoinData.drop(columns=predictors).iloc[:-1000]
y = bitcoinData['target'].iloc[:-1000]

x_test = bitcoinData.drop(columns=predictors).iloc[-1000:]
y_test = bitcoinData['target'].iloc[-1000:]

feature_list = x.to_dict(orient='records')
target_list = y.tolist()

# # x_iter = x.to_L
# y_iter = y.items()

# x_slice = itertools.islice(x_iter, 1000)
# y_slice = itertools.islice(y_iter, 1000)

bitcoinData

C:\Users\shand\AppData\Local\Temp\ipykernel_20568\2865747372.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoinData.drop(columns=['n','Volume_(BTC)', 'vw'], inplace=True)
C:\Users\shand\AppData\Local\Temp\ipykernel_20568\2865747372.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoinData["next_minute"] = bitcoinData["Close"].shift(-1)
C:\Users\shand\AppData\Local\Temp\ipykernel_20568\2865747372.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Close,High,Low,Open,next_minute,target,Up,Close_Ratio_1,Trend_1,Close_Ratio_2,...,Close_Ratio_6,Trend_6,Close_Ratio_7,Trend_7,Close_Ratio_8,Trend_8,Close_Ratio_9,Trend_9,Close_Ratio_10,Trend_10
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-08-01 00:10:00,29214.180870,29220.268000,29207.460814,29207.460814,29214.718918,1,1,1.0,0.0,0.999963,...,1.000094,3.0,1.000035,3.0,0.999979,4.0,0.999948,4.0,0.999908,5.0
2023-08-01 00:11:00,29214.718918,29218.844119,29207.411055,29218.844119,29217.338347,1,1,1.0,1.0,1.000009,...,1.000075,4.0,1.000096,4.0,1.000047,4.0,0.999998,5.0,0.999970,5.0
2023-08-01 00:12:00,29217.338347,29223.530000,29209.976000,29214.745084,29216.869740,0,0,1.0,1.0,1.000045,...,1.000118,4.0,1.000141,5.0,1.000163,5.0,1.000121,5.0,1.000079,6.0
2023-08-01 00:13:00,29216.869740,29222.344228,29210.874045,29212.796303,29218.431150,1,1,1.0,0.0,0.999992,...,1.000067,3.0,1.000088,4.0,1.000109,5.0,1.000130,5.0,1.000095,5.0
2023-08-01 00:14:00,29218.431150,29221.869131,29212.620805,29221.869131,29224.597401,1,1,1.0,1.0,1.000027,...,1.000072,4.0,1.000104,4.0,1.000123,5.0,1.000145,6.0,1.000165,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-07 22:48:00,29167.760050,29170.877780,29162.225000,29168.870291,29165.602111,0,0,1.0,0.0,0.999971,...,1.000205,3.0,1.000233,3.0,1.000249,4.0,1.000269,5.0,1.000297,5.0
2023-08-07 22:49:00,29165.602111,29170.623488,29160.115000,29161.795862,29160.392200,0,0,1.0,0.0,0.999963,...,1.000119,2.0,1.000113,3.0,1.000139,3.0,1.000155,4.0,1.000175,5.0
2023-08-07 22:50:00,29160.392200,29170.867712,29157.394522,29164.651736,29160.917934,1,1,1.0,0.0,0.999911,...,0.999894,2.0,0.999949,2.0,0.999942,3.0,0.999965,3.0,0.999979,4.0


In [24]:
import itertools as IT
dataset = df.iloc[:-1000]

model = forest.AMFClassifier(
    n_estimators=250,
    use_aggregation=True,
    dirichlet=5,
    seed=199
)

instance_stream = stream.iter_array(feature_list, target_list)

metric = metrics.Accuracy()

evaluate.progressive_val_score(instance_stream, model, metric)

Accuracy: 49.56%

In [25]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=250,min_samples_split=75, random_state=1)
train = bitcoinData[:-200]
test = bitcoinData[-200:]

model.fit(train[predictors], train["Up"])

from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["Up"], preds)

0.5531914893617021

In [26]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    
    return pd.concat(all_predictions)

In [27]:
predictions = backtest(bitcoinData, model, predictors)

In [29]:
predictions["Predictions"].value_counts()
precision_score(predictions["target"], predictions["Predictions"])

0.5690353958389625

In [30]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= .6] = 1
    preds[preds < .6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [31]:
predictions = backtest(bitcoinData, model, predictors)

In [32]:
predictions["Predictions"].value_counts()
precision_score(predictions["target"], predictions["Predictions"])

0.6263924592973437

In [33]:
model.fit(bitcoinData[predictors], bitcoinData["target"])

RandomForestClassifier(min_samples_split=75, n_estimators=250, random_state=1)

In [ ]:
import pickle

with open("bittybotV2.pkl", 'wb') as f:
    pickle.dump(model,f)